# Nurse Scheduling Problem

Lucerne University of Applied Sciences and Arts - School of Information Technology

A hospital supervisor needs to create a weekly schedule for 4 nurses:
* Each day is divided into three 8-hour shifts
* A nurse must not work two shifts on the same day
* Each nurse works five or six days a week
* No shift is staffed by more than two different nurses in a week
* If a nurse works shifts 2 or 3 on a given day, (s)he must also work the same shift either the previous day or the following day (does not apply to holidays)


@author: Marc Pouly and Tobias Mérinat

In [1]:
from ortools.sat.python import cp_model

Nurse Schedule Problem from lecture

In [2]:
# 4 nurses
num_nurses = 4
# 7 days a week
num_days = 7
# 4 shifts (holiday_shift = 0, day_shift = 1, late_shift = 2, night_shift = 3)
num_shifts = 4

Create constraint solver

In [3]:
model = cp_model.CpModel()

In [4]:
# Matrix of Boolean decision variables: NURSES x DAYS x SHIFTS
# schedule[s][d][n] = 1 means nurse n works on day d in shift s 

schedule = [[[model.NewBoolVar(f"({_n},{_d},{_s})") 
              for _n in range(num_nurses)] 
              for _d in range(num_days)] 
              for _s in range(num_shifts)]

print(f"Shifts = {len(schedule)}, Days = {len(schedule[0])}, Nurses = {len(schedule[0][0])}")
print(schedule)

Shifts = 4, Days = 7, Nurses = 4
[[[(0,0,0)(0..1), (1,0,0)(0..1), (2,0,0)(0..1), (3,0,0)(0..1)], [(0,1,0)(0..1), (1,1,0)(0..1), (2,1,0)(0..1), (3,1,0)(0..1)], [(0,2,0)(0..1), (1,2,0)(0..1), (2,2,0)(0..1), (3,2,0)(0..1)], [(0,3,0)(0..1), (1,3,0)(0..1), (2,3,0)(0..1), (3,3,0)(0..1)], [(0,4,0)(0..1), (1,4,0)(0..1), (2,4,0)(0..1), (3,4,0)(0..1)], [(0,5,0)(0..1), (1,5,0)(0..1), (2,5,0)(0..1), (3,5,0)(0..1)], [(0,6,0)(0..1), (1,6,0)(0..1), (2,6,0)(0..1), (3,6,0)(0..1)]], [[(0,0,1)(0..1), (1,0,1)(0..1), (2,0,1)(0..1), (3,0,1)(0..1)], [(0,1,1)(0..1), (1,1,1)(0..1), (2,1,1)(0..1), (3,1,1)(0..1)], [(0,2,1)(0..1), (1,2,1)(0..1), (2,2,1)(0..1), (3,2,1)(0..1)], [(0,3,1)(0..1), (1,3,1)(0..1), (2,3,1)(0..1), (3,3,1)(0..1)], [(0,4,1)(0..1), (1,4,1)(0..1), (2,4,1)(0..1), (3,4,1)(0..1)], [(0,5,1)(0..1), (1,5,1)(0..1), (2,5,1)(0..1), (3,5,1)(0..1)], [(0,6,1)(0..1), (1,6,1)(0..1), (2,6,1)(0..1), (3,6,1)(0..1)]], [[(0,0,2)(0..1), (1,0,2)(0..1), (2,0,2)(0..1), (3,0,2)(0..1)], [(0,1,2)(0..1), (1,1,2)(0..1), 

In [5]:
# Every shift must have exactly one nurse assigned on each day

for s in range(num_shifts):
    for d in range(num_days):
        # Day and shift being fixed, the number of nurses must be exactly 1
        model.Add(sum([schedule[s][d][n] for n in range(num_nurses)]) == 1)

In [6]:
# A nurse must not work two shifts on the same day

for n in range(num_nurses):
    for d in range(num_days):
        # Nurse and day being fixed, the number of shifts can be at most 1
        model.Add(sum([schedule[s][d][n] for s in range(num_shifts)]) < 2)

In [7]:
# Each nurse has one or two days off

for n in range(num_nurses):
    # Nurse and shift = 0 (holidays) being fixed, the number of days is either 1 or 2
    num_days_off = sum([schedule[0][d][n] for d in range(num_days)])
    model.Add(num_days_off > 0)
    model.Add(num_days_off < 3)

In [8]:
# Each shift except shift 0 is staffed by at most two nurses per week

for s in range(1, num_shifts):
    # Shift being fixed, memorize whether a nurse works in this shift
    does_work = [model.NewBoolVar('') for _n in range(num_nurses)]
    # make sure does_work respects given rules:
    for n in range(num_nurses):
        # Count the number of days the current nurse works in the current shift
        var = sum([schedule[s][d][n] for d in range(num_days)])
        # If nurse works in current shift, set variable to true
        model.Add(var > 0).OnlyEnforceIf(does_work[n])
        # If nurse does not work in current shift, set variable to false
        model.Add(var == 0).OnlyEnforceIf(does_work[n].Not())    
    # The number of nurses who work in current shift must be 2
    model.Add(sum(does_work) <= 2)     

In [9]:
# For shift 2 and 3
for s in [2,3]:
    for n in range(num_nurses):
        for d in range(num_days):
            # Does current nurse work either the day before or after?
            before_or_after = model.NewBoolVar('')
            model.AddMaxEquality(before_or_after, [schedule[s][(d-1) % num_days][n], schedule[s][(d+1) % num_days][n]])
            # When current nurse works on current day, (s)he must work either day before or after
            model.Add(before_or_after == 1).OnlyEnforceIf(schedule[s][d][n])

In [10]:
nurses_with_more_than_1d_off = [model.NewBoolVar(str(c)) for c in range(num_nurses)]

# Count how many days off a nurse has
for n in range(num_nurses):
    num_days_off = sum([schedule[0][d][n] for d in range(num_days)])
    # When count > 1 the nurse has two days off
    model.Add(num_days_off == 2).OnlyEnforceIf(nurses_with_more_than_1d_off[n])
    # When count == 1 the nurse has only one day off
    model.Add(num_days_off == 1).OnlyEnforceIf(nurses_with_more_than_1d_off[n].Not())

model.Maximize(sum(nurses_with_more_than_1d_off))


In [11]:
class MaxNSolutionPrinter(cp_model.CpSolverSolutionCallback):
        
    def __init__(self, nb_sol, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__nb_sol = nb_sol
        self.__counter = 0
        
    def on_solution_callback(self):
        self.__counter += 1
        if self.__counter <= self.__nb_sol:
            self.pretty_print(self.__variables)
            
    def solution_count(self):
        return self.__counter
            
    def pretty_print(self, schedule):
        mapper1 = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
        mapper2 = {0: 'H  |', 1: 'S1 |', 2: 'S2 |', 3: 'S3 |'}
        print("SHIFT | M T W T F S S")
        print("---------------------")
        for s in range(num_shifts):
            print(f"   {mapper2[s]} ", end = '')
            for d in range(num_days):
                for n in range(num_nurses):
                    if self.Value(schedule[s][d][n]) == 1:
                        print(f"{mapper1[n]} ", end = '')
            print("")
        print("\n")  

In [12]:
solver = cp_model.CpSolver()
callback = MaxNSolutionPrinter(1, schedule)
status = solver.SearchForAllSolutions(model, callback)

print("Statistics")
print(f" - Conflicts: \t{solver.NumConflicts()}")
print(f" - Branches: \t{solver.NumBranches()}")
print(f" - Walltime: \t{solver.WallTime():.2f}s")
print(f" - Solutions: \t{callback.solution_count()}")

TypeError: Search for all solutions is only defined on satisfiability problems